<a href="https://colab.research.google.com/github/jtmeawno1/BasicProg_DataViz65/blob/main/midterm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1 ทำนายราคาบ้าน 10 คะแนน
เตรียมข้อมูลเพื่อทำนายราคาบ้าน ราคาประเมินบ้าน และ คะแนนทรัพย์ จาก [google drive](https://drive.google.com/drive/folders/1uIcUFY1yldIq-A3wijh2n2N6az-EljIe?usp=sharing) โดยข้อมูลที่เตรียมต้องประกอบด้วย columns ต่อไปนี้

* ```'renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking', 'sell_price'``` จากตาราง "คำตอบนักประเมิน(ID)"

* ```จำนวน 7-11 และ โรงเรียน ในรัศมี 200,500,1000,2000 เมตร (ตั้งชื่อ d200-7/11','d500-7/11',...,'d2000-7/11' แบะ ;d200-school','d500-school',...,'d2000-school')``` จากตาราง "7-eleven.json" และ "school.json"

* ```ประเมินราคาซื้อขาย และ คะแนนทรัพย์ (ตั้งชื่อ 'estimator' และ 'score')``` จากตาราง คำตอบนักประเมิน(ID) โดยราคาประเมินกับคะแนนทรัพย์ อ้างอิงของนักประเมิน ID=01 คนเดียว

In [1]:
import pandas as pd
import numpy as np
from pandas.io import json
import matplotlib.pyplot as plt
import seaborn as sns
import geopy.distance

In [2]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


#import file  

In [3]:
#คำตอบนักประเมิน(ID)
link1 = 'https://docs.google.com/spreadsheets/d/1F5Og-SUPEbjdMfIjyvsYkKE4tgvvOV6E/export?format=xlsx'
detachedhouse = pd.read_excel(link1,sheet_name='บ้าน')
secondhand = pd.read_excel(link1,sheet_name='secondhand')

#Data House
link2 = 'https://docs.google.com/spreadsheets/d/1UJA8RJhC20SVg0qcBs1iy2lLYk2_a8e7/export?format=xlsx'
H1 = pd.read_excel(link2,sheet_name='บ้านมือ1')
H2 = pd.read_excel(link2,sheet_name='บ้านมือ2')

#7-eleven.json
with open('/content/drive/MyDrive/MIDterm/Midterm/7-eleven.json') as json_file:
  this_json_seven = json.loads(json_file.read())

#school.json
with open('/content/drive/MyDrive/MIDterm/Midterm/school.json') as json_file:
  this_jsonschool = json.loads(json_file.read())

# data บ้าน

## บ้านมือ 1

In [4]:
  H1 =H1[['id','latitude','longitude','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking','sell_price']]

### แก้ missing

In [5]:
H1.isnull().sum()

id                      0
latitude                9
longitude               9
number_of_rai         754
number_of_ngan        754
number_of_va           92
number_of_story         4
number_of_bedroom      10
number_of_bathroom     10
direction              88
furniture              88
usable_area            36
number_of_parking      21
sell_price             21
dtype: int64

In [6]:
#fill NA with 0
H1 = H1.fillna(0)

In [7]:
# แก้ id ให้เป็นตัวเลข
#data['result'] = data['result'].map(lambda x: x.lstrip('+-').rstrip('aAbBcC'))
H1['id'] = H1['id'].apply(str).str.replace('project_', '')

In [8]:
H1.tail(5)

,id,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price
749,62204519412ede001495e6cb,102.875722,16.484111,0.0,0.0,0.0,1.0,3.0,2.0,0.0,0.0,125.0,2.0,2290000.0
750,62201e6fa441d30014e1f350,102.850706,16.410972,0.0,0.0,0.0,1.0,3.0,2.0,0.0,0.0,70.0,2.0,2590000.0
751,5fa0e476a3326800191e29ea,102.614819,15.828671,0.0,0.0,0.0,1.0,3.0,2.0,0.0,0.0,120.0,2.0,1990000.0
752,5e9d6fa69f161600121b61b2,102.402141,15.808013,0.0,0.0,0.0,1.0,3.0,2.0,0.0,0.0,100.0,2.0,1990000.0
753,5ea938d968253d0012adc1e2,102.722853,16.425078,0.0,0.0,0.0,1.0,3.0,2.0,0.0,0.0,138.0,2.0,2490000.0


In [9]:
for i in H1.columns:
    print(i)

id
latitude
longitude
number_of_rai
number_of_ngan
number_of_va
number_of_story
number_of_bedroom
number_of_bathroom
direction
furniture
usable_area
number_of_parking
sell_price


## บ้านมือ 2

In [10]:
H2 =H2[['id','latitude','longitude','renovate','number_of_rai','number_of_ngan','number_of_va','number_of_story','number_of_bedroom','number_of_bathroom','direction','furniture','usable_area','number_of_parking','sell_price']]

###  แก้ missing

In [11]:
H2.isnull().sum()

id                      0
latitude               22
longitude              22
renovate              211
number_of_rai           0
number_of_ngan          0
number_of_va            0
number_of_story       549
number_of_bedroom      27
number_of_bathroom      0
direction               0
furniture               0
usable_area           920
number_of_parking     215
sell_price              1
dtype: int64

In [12]:
# แก้ missing ด้วย fill NA with 0
H2.fillna(0)

,id,latitude,longitude,renovate,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,direction,furniture,usable_area,number_of_parking,sell_price
0,1.0,16.139321,102.631587,no,0.0,1.0,45.0,2.0,2.0,2.0,ไม่ระบุ,ไม่มี,244.00,2.0,390000.0
1,4.0,16.676156,102.917998,no,0.0,0.0,88.0,1.0,1.0,1.0,ไม่ระบุ,ไม่มี,164.00,3.0,820000.0
2,16.0,16.129358,102.530670,no,1.0,0.0,8.0,1.0,11.0,11.0,ไม่ระบุ,ไม่ระบุ,489.00,9.0,3260000.0
3,18.0,16.459052,102.806404,no,0.0,0.0,61.6,2.0,3.0,3.0,ไม่ระบุ,ไม่ระบุ,203.00,2.0,5000000.0
4,19.0,16.450000,102.800680,no,0.0,0.0,72.3,2.0,3.0,2.0,ไม่ระบุ,ไม่ระบุ,164.00,2.0,3500000.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1651,20460P,16.480110,102.574033,no,0.0,0.0,73.0,1.0,2.0,1.0,ไม่ระบุ,ไม่ระบุ,108.76,0.0,437400.0
1652,20462P,16.528551,102.411762,no,0.0,0.0,84.0,1.0,2.0,1.0,ไม่ระบุ,ไม่ระบุ,119.00,0.0,714600.0
1653,20463P,15.867444,102.482085,no,0.0,1.0,45.0,2.0,1.0,1.0,ไม่ระบุ,ไม่ระบุ,188.50,0.0,451000.0
1654,20466P,16.636912,102.105343,no,0.0,1.0,59.0,2.0,1.0,1.0,ไม่ระบุ,ไม่ระบุ,207.00,0.0,910000.0


###  แก้  value ให้ตรงกัน

In [13]:
# ลบ P จาก id
H2['id'] = H2['id'].apply(str).str.replace('P', '')

In [14]:
# ลบ ของเมือง, ทางทิศ, ทิศ จาก ตะวันออกเฉียงใต้ของเมือง, ทางทิศตะวันออก, ทิศตะวันออก, ทิศเหนือ
H2['direction'] = H2['direction'].apply(str).str.replace('ของเมือง', '')
H2['direction'] = H2['direction'].apply(str).str.replace('ทางทิศ', '')
H2['direction'] = H2['direction'].apply(str).str.replace('ทิศ', '')

### one-hot-encoder 

In [15]:
#one-hot-encoder 
# ทำ columns 'renovate','direction','furniture' เป็น dummy
H2= pd.get_dummies(H2,  columns=['renovate','direction','furniture'])

## รวมตาราง 

In [16]:
data_house = pd.concat([H1, H2], keys=['1', '2']).reset_index() # รวมตารางบ้านมือ1กับมือ2 แนวแกนy keys คือสร้างคอลัมน์ second_hand เพื่อระบุว่า Row แต่ละอันมาจากบ้านมือ1หรือมือ2
data_house.rename(index=str, columns={'level_0': 'second_hand'}, inplace=True)  #rename เป็น second_hand
data_house.drop(['level_1'], axis=1, inplace=True) # drop col index

In [17]:
data_house = data_house.fillna(0)

In [18]:
data_house

,second_hand,id,latitude,longitude,number_of_rai,number_of_ngan,number_of_va,number_of_story,number_of_bedroom,number_of_bathroom,...,direction_ตะวันออก,direction_ตะวันออกเฉียงเหนือ,direction_ตะวันออกเฉียงใต้,direction_เหนือ,direction_ใต้,direction_ไม่ระบุ,furniture_ตกแต่งครบ,furniture_มีบางส่วน,furniture_ไม่มี,furniture_ไม่ระบุ
0,1,314.0,16.392304,102.827926,0.0,0.0,50.00,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,315.0,16.392304,102.827926,0.0,0.0,40.00,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1,316.0,16.392304,102.827926,0.0,0.0,77.25,2.0,4.0,3.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1,317.0,16.392304,102.827926,0.0,0.0,81.25,2.0,6.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1,318.0,16.392304,102.827926,0.0,0.0,55.00,2.0,4.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2405,2,20460,16.480110,102.574033,0.0,0.0,73.00,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2406,2,20462,16.528551,102.411762,0.0,0.0,84.00,1.0,2.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2407,2,20463,15.867444,102.482085,0.0,1.0,45.00,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
2408,2,20466,16.636912,102.105343,0.0,1.0,59.00,2.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0


# data เซเว่น

In [40]:
svid = pd.DataFrame.from_dict(seven_id,orient='index') #เรียกข้อมูลเซเว่นให้มาอยู่ในรูปของดาต้าเฟรม

In [41]:
svid_reset = svid.reset_index() # reset index เพื่อให้คอลลัมน์แรกเป็นจำนวนเรคคอร์ด

In [42]:
svid_changename = svid_reset.rename(columns={'index':'seven_id'})

In [43]:
svid_changename #เปลี่ยนชื่อหัวตารางจาก index เป็น seven id

,seven_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,6,"{'lat': 16.0603149, 'lng': 102.7264856}","{'lat': 16.0564741, 'lng': 102.7268436}","{'lat': 16.0595413, 'lng': 102.7294307}","{'lat': 16.0751998, 'lng': 102.7356017}","{'lat': 16.070648, 'lng': 102.7344055}","{'lat': 16.0582995, 'lng': 102.7360953}","{'lat': 16.0573901, 'lng': 102.7192322}",None,None,...,None,None,None,None,None,None,None,None,None,None
1,11,"{'lat': 15.809535, 'lng': 102.6069684}","{'lat': 15.8193062, 'lng': 102.6012589}","{'lat': 15.8125848, 'lng': 102.6009202}","{'lat': 15.8102763, 'lng': 102.5985417}","{'lat': 15.8095537, 'lng': 102.5879272}","{'lat': 15.7970035, 'lng': 102.6075296}",None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,16,"{'lat': 16.1340329, 'lng': 102.5331697}","{'lat': 16.1294298, 'lng': 102.540669}",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,17,"{'lat': 16.4052438, 'lng': 102.7823961}","{'lat': 16.3898169, 'lng': 102.8053144}","{'lat': 16.4221806, 'lng': 102.8047853}","{'lat': 16.4083692, 'lng': 102.7948258}","{'lat': 16.4053255, 'lng': 102.7746538}",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,18,"{'lat': 16.4506978, 'lng': 102.7955315}","{'lat': 16.445794, 'lng': 102.8242206}","{'lat': 16.4410003, 'lng': 102.8216208}","{'lat': 16.4600404, 'lng': 102.8247777}","{'lat': 16.4421359, 'lng': 102.8198991}","{'lat': 16.4512089, 'lng': 102.795423}","{'lat': 16.4423, 'lng': 102.808299}","{'lat': 16.464662, 'lng': 102.8251211}","{'lat': 16.4656851, 'lng': 102.8249423}",...,"{'lat': 16.4622829, 'lng': 102.8270049}","{'lat': 16.4781927, 'lng': 102.8199828}","{'lat': 16.4777682, 'lng': 102.8235898}","{'lat': 16.4786388, 'lng': 102.814293}","{'lat': 16.4740341, 'lng': 102.8262668}",None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694,30332P,"{'lat': 16.4053255, 'lng': 102.7746538}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2695,30334P,"{'lat': 16.4685967, 'lng': 102.8296805}","{'lat': 16.4866589, 'lng': 102.8158198}","{'lat': 16.4642871, 'lng': 102.8283136}","{'lat': 16.4794698, 'lng': 102.8070934}","{'lat': 16.4908451, 'lng': 102.8321155}","{'lat': 16.4884547, 'lng': 102.8248678}","{'lat': 16.4781927, 'lng': 102.8199828}","{'lat': 16.4786388, 'lng': 102.814293}","{'lat': 16.4821349, 'lng': 102.8194465}",...,"{'lat': 16.4627525, 'lng': 102.8277724}","{'lat': 16.4671081, 'lng': 102.8297358}","{'lat': 16.4863874, 'lng': 102.8164751}","{'lat': 16.464662, 'lng': 102.8251211}","{'lat': 16.4802561, 'lng': 102.8118345}","{'lat': 16.4622829, 'lng': 102.8270049}","{'lat': 16.4644021, 'lng': 102.8275917}","{'lat': 16.4777682, 'lng': 102.8235898}","{'lat': 16.4800345, 'lng': 102.817991}","{'lat': 16.4834067, 'lng': 102.832897}"
2696,30359P,"{'lat': 16.4506978, 'lng': 102.7955315}","{'lat': 16.4342316, 'lng': 102.8145643}","{'lat': 16.4362708, 'lng': 102.7997081}","{'lat': 16.4379979, 'lng': 102.7868055}","{'lat': 16.4421359, 'lng': 102.8198991}","{'lat': 16.4423, 'lng': 102.808299}","{'lat': 16.4212509, 'lng': 102.8156687}","{'lat': 16.4221806, 'lng': 102.8047853}","{'lat': 16.4463944, 'lng': 102.8200097}",...,"{'lat': 16.4215105, 'lng': 102.8133801}","{'lat': 16.4210166, 'lng': 102.8099416}","{'lat': 16.4213278, 'lng': 102.8185574}",None,None,None,None,None,None,None
2697,30360P,"{'lat': 16.4685967, 'lng': 102.8296805}","{'lat': 16.4866589, 'lng': 102.8158198}","{'lat': 16.4794698, 'lng': 102.8070934}","{'lat': 16.4908451, 'lng': 102.8321155}","{'lat': 16.4884547, 'lng': 102.8248678}","{'lat': 16.4781927, 'lng': 102.8199828}","{'lat': 16.4786388, 'lng': 102.814293}","{'lat': 16.4821349, 'lng': 102.8194465}","{'lat': 16.4671081, 'lng': 102.8297358}",...,"{'lat': 16.4802561, 'lng': 102.8118345}","{'lat': 16.4777682, 'lng': 102.8235898}","{'lat': 16.4800345, 'lng': 102.817991}","{'lat': 16.4834067, 'lng': 102.832897}","{'lat': 16.4682

In [44]:
svid_changename['seven_id'].isna().sum()

0

In [45]:
svid_changename.astype(str)

,seven_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,6,"{'lat': 16.0603149, 'lng': 102.7264856}","{'lat': 16.0564741, 'lng': 102.7268436}","{'lat': 16.0595413, 'lng': 102.7294307}","{'lat': 16.0751998, 'lng': 102.7356017}","{'lat': 16.070648, 'lng': 102.7344055}","{'lat': 16.0582995, 'lng': 102.7360953}","{'lat': 16.0573901, 'lng': 102.7192322}",None,None,...,None,None,None,None,None,None,None,None,None,None
1,11,"{'lat': 15.809535, 'lng': 102.6069684}","{'lat': 15.8193062, 'lng': 102.6012589}","{'lat': 15.8125848, 'lng': 102.6009202}","{'lat': 15.8102763, 'lng': 102.5985417}","{'lat': 15.8095537, 'lng': 102.5879272}","{'lat': 15.7970035, 'lng': 102.6075296}",None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,16,"{'lat': 16.1340329, 'lng': 102.5331697}","{'lat': 16.1294298, 'lng': 102.540669}",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,17,"{'lat': 16.4052438, 'lng': 102.7823961}","{'lat': 16.3898169, 'lng': 102.8053144}","{'lat': 16.4221806, 'lng': 102.8047853}","{'lat': 16.4083692, 'lng': 102.7948258}","{'lat': 16.4053255, 'lng': 102.7746538}",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,18,"{'lat': 16.4506978, 'lng': 102.7955315}","{'lat': 16.445794, 'lng': 102.8242206}","{'lat': 16.4410003, 'lng': 102.8216208}","{'lat': 16.4600404, 'lng': 102.8247777}","{'lat': 16.4421359, 'lng': 102.8198991}","{'lat': 16.4512089, 'lng': 102.795423}","{'lat': 16.4423, 'lng': 102.808299}","{'lat': 16.464662, 'lng': 102.8251211}","{'lat': 16.4656851, 'lng': 102.8249423}",...,"{'lat': 16.4622829, 'lng': 102.8270049}","{'lat': 16.4781927, 'lng': 102.8199828}","{'lat': 16.4777682, 'lng': 102.8235898}","{'lat': 16.4786388, 'lng': 102.814293}","{'lat': 16.4740341, 'lng': 102.8262668}",None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2694,30332P,"{'lat': 16.4053255, 'lng': 102.7746538}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2695,30334P,"{'lat': 16.4685967, 'lng': 102.8296805}","{'lat': 16.4866589, 'lng': 102.8158198}","{'lat': 16.4642871, 'lng': 102.8283136}","{'lat': 16.4794698, 'lng': 102.8070934}","{'lat': 16.4908451, 'lng': 102.8321155}","{'lat': 16.4884547, 'lng': 102.8248678}","{'lat': 16.4781927, 'lng': 102.8199828}","{'lat': 16.4786388, 'lng': 102.814293}","{'lat': 16.4821349, 'lng': 102.8194465}",...,"{'lat': 16.4627525, 'lng': 102.8277724}","{'lat': 16.4671081, 'lng': 102.8297358}","{'lat': 16.4863874, 'lng': 102.8164751}","{'lat': 16.464662, 'lng': 102.8251211}","{'lat': 16.4802561, 'lng': 102.8118345}","{'lat': 16.4622829, 'lng': 102.8270049}","{'lat': 16.4644021, 'lng': 102.8275917}","{'lat': 16.4777682, 'lng': 102.8235898}","{'lat': 16.4800345, 'lng': 102.817991}","{'lat': 16.4834067, 'lng': 102.832897}"
2696,30359P,"{'lat': 16.4506978, 'lng': 102.7955315}","{'lat': 16.4342316, 'lng': 102.8145643}","{'lat': 16.4362708, 'lng': 102.7997081}","{'lat': 16.4379979, 'lng': 102.7868055}","{'lat': 16.4421359, 'lng': 102.8198991}","{'lat': 16.4423, 'lng': 102.808299}","{'lat': 16.4212509, 'lng': 102.8156687}","{'lat': 16.4221806, 'lng': 102.8047853}","{'lat': 16.4463944, 'lng': 102.8200097}",...,"{'lat': 16.4215105, 'lng': 102.8133801}","{'lat': 16.4210166, 'lng': 102.8099416}","{'lat': 16.4213278, 'lng': 102.8185574}",None,None,None,None,None,None,None
2697,30360P,"{'lat': 16.4685967, 'lng': 102.8296805}","{'lat': 16.4866589, 'lng': 102.8158198}","{'lat': 16.4794698, 'lng': 102.8070934}","{'lat': 16.4908451, 'lng': 102.8321155}","{'lat': 16.4884547, 'lng': 102.8248678}","{'lat': 16.4781927, 'lng': 102.8199828}","{'lat': 16.4786388, 'lng': 102.814293}","{'lat': 16.4821349, 'lng': 102.8194465}","{'lat': 16.4671081, 'lng': 102.8297358}",...,"{'lat': 16.4802561, 'lng': 102.8118345}","{'lat': 16.4777682, 'lng': 102.8235898}","{'lat': 16.4800345, 'lng': 102.817991}","{'lat': 16.4834067, 'lng': 102.832897}","{'lat': 16.4682

ถ้าต้องเปลี่ยนไอดีให้อยู่ในลักษณะเลขที่รันได้ ก็ต้องแก้ regular expression

In [46]:
svid_changetype = svid_changename['seven_id'].str.replace("P","")  

In [47]:
svid_changetype

0           6
1          11
2          16
3          17
4          18
        ...  
2694    30332
2695    30334
2696    30359
2697    30360
2698    30362
Name: seven_id, Length: 2699, dtype: object

In [48]:
seven = pd.merge(svid_changetype,svid_changename)

In [50]:
seven

,seven_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,6,"{'lat': 16.0603149, 'lng': 102.7264856}","{'lat': 16.0564741, 'lng': 102.7268436}","{'lat': 16.0595413, 'lng': 102.7294307}","{'lat': 16.0751998, 'lng': 102.7356017}","{'lat': 16.070648, 'lng': 102.7344055}","{'lat': 16.0582995, 'lng': 102.7360953}","{'lat': 16.0573901, 'lng': 102.7192322}",None,None,...,None,None,None,None,None,None,None,None,None,None
1,11,"{'lat': 15.809535, 'lng': 102.6069684}","{'lat': 15.8193062, 'lng': 102.6012589}","{'lat': 15.8125848, 'lng': 102.6009202}","{'lat': 15.8102763, 'lng': 102.5985417}","{'lat': 15.8095537, 'lng': 102.5879272}","{'lat': 15.7970035, 'lng': 102.6075296}",None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,16,"{'lat': 16.1340329, 'lng': 102.5331697}","{'lat': 16.1294298, 'lng': 102.540669}",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,17,"{'lat': 16.4052438, 'lng': 102.7823961}","{'lat': 16.3898169, 'lng': 102.8053144}","{'lat': 16.4221806, 'lng': 102.8047853}","{'lat': 16.4083692, 'lng': 102.7948258}","{'lat': 16.4053255, 'lng': 102.7746538}",None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,18,"{'lat': 16.4506978, 'lng': 102.7955315}","{'lat': 16.445794, 'lng': 102.8242206}","{'lat': 16.4410003, 'lng': 102.8216208}","{'lat': 16.4600404, 'lng': 102.8247777}","{'lat': 16.4421359, 'lng': 102.8198991}","{'lat': 16.4512089, 'lng': 102.795423}","{'lat': 16.4423, 'lng': 102.808299}","{'lat': 16.464662, 'lng': 102.8251211}","{'lat': 16.4656851, 'lng': 102.8249423}",...,"{'lat': 16.4622829, 'lng': 102.8270049}","{'lat': 16.4781927, 'lng': 102.8199828}","{'lat': 16.4777682, 'lng': 102.8235898}","{'lat': 16.4786388, 'lng': 102.814293}","{'lat': 16.4740341, 'lng': 102.8262668}",None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1840,24178,"{'lat': 16.4481776, 'lng': 102.7736213}","{'lat': 16.4496233, 'lng': 102.7578029}",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1841,24179,"{'lat': 16.4908451, 'lng': 102.8321155}","{'lat': 16.525684, 'lng': 102.8320249}","{'lat': 16.4977439, 'lng': 102.8364985}","{'lat': 16.4884547, 'lng': 102.8248678}",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1842,24180,"{'lat': 16.4229267, 'lng': 102.8712313}","{'lat': 16.4251666, 'lng': 102.8510168}","{'lat': 16.4365317, 'lng': 102.8491264}","{'lat': 16.4285371, 'lng': 102.8496085}","{'lat': 16.4198307, 'lng': 102.8496996}","{'lat': 16.434367, 'lng': 102.8609234}","{'lat': 16.4236724, 'lng': 102.8434961}","{'lat': 16.4289683, 'lng': 102.8471979}","{'lat': 16.4288908, 'lng': 102.8559018}",...,"{'lat': 16.4288377, 'lng': 102.8542497}","{'lat': 16.4276748, 'lng': 102.8632914}","{'lat': 16.4320627, 'lng': 102.8688061}","{'lat': 16.4289572, 'lng': 102.8477644}","{'lat': 16.448722, 'lng': 102.841044}","{'lat': 16.4321944, 'lng': 102.8410091}","{'lat': 16.4255945, 'lng': 102.8397278}","{'lat': 16.4533508, 'lng': 102.845216}","{'lat': 16.4187106, 'lng': 102.8405019}",None
1843,24181,"{'lat': 16.4510959, 'lng': 102.6397602}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


In [57]:
for id in seven['seven_id']:
 print(id)

6
11
16
17
18
19
20
21
22
23
24
25
26
28
29
30
31
32
33
35
36
40
491
493
495
497
498
499
508
509
510
511
512
513
515
517
518
519
520
521
522
523
527
528
531
532
533
534
536
538
540
541
542
543
545
547
548
549
550
551
552
553
554
555
557
558
560
561
562
563
564
565
566
567
568
569
570
571
572
573
574
575
576
577
578
579
580
581
582
583
584
585
586
587
588
589
590
591
592
593
594
595
596
597
598
600
601
602
603
604
605
606
607
608
609
610
611
612
613
614
615
616
617
618
619
620
621
622
623
624
625
626
627
629
631
632
633
634
635
636
637
638
640
641
642
643
644
645
646
647
648
649
650
651
652
653
654
655
656
657
658
660
661
662
663
664
665
666
667
668
669
670
671
672
673
674
675
676
677
678
679
681
682
683
684
685
686
687
688
689
690
691
692
693
694
695
696
697
698
699
700
701
702
703
704
705
706
707
708
709
710
711
712
713
714
717
720
758
759
760
761
1024
1024
1033
1033
1057
1057
1058
1058
1062
1063
1063
1064
1065
1065
1066
1066
1067
1068
1068
1070
1070
1071
1071
1074
1074
1084
1084
1088

In [66]:
# กำหนด lat lng ของบ้านก่อน
h_lat = data_house['latitude'].values[0]
h_long = data_house['longitude'].values[0]

In [73]:
thisseven = seven_id[str(id)]

KeyError: ignored

In [70]:
seven_dist=list() #listว่างเพราะจะทำมาเก็บ 
loc1 = [h_lat,h_long] #เอาไว้นอกลูป เพราะแผนที่บ้านเราไม่ได้วนในลูป
for seven in thisseven:
  loc2 = [seven['lat'],seven['lng']]
  dist = geopy.distance.geodesic(loc1,loc2).m
  print(dist) #ลองปริ้นออกมาดูว่ามีระยะห่างเท่าไหร่ 
  seven_dist.append(dist)

NameError: ignored

# data โรงเรียน


In [ ]:
scid = pd.DataFrame.from_dict(school_id,orient='index') #เรียกข้อมูลเซเว่นให้มาอยู่ในรูปของดาต้าเฟรม

In [ ]:
scid_reset = scid.reset_index() # reset index เพื่อให้คอลลัมน์แรกเป็นจำนวนเรคคอร์ด

In [ ]:
scid_changename = scid_reset.rename(columns={'index':'school_idno'})

In [ ]:
scid_changename

,school_idno,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,1,"{'lat': 16.1311875, 'lng': 102.6504375}","{'lat': 16.1350641, 'lng': 102.6290474}","{'lat': 16.1482073, 'lng': 102.6207588}","{'lat': 16.1279195, 'lng': 102.6138295}",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,4,"{'lat': 16.6744524, 'lng': 102.9189581}","{'lat': 16.6934, 'lng': 102.926531}","{'lat': 16.682199, 'lng': 102.909279}","{'lat': 16.6876442, 'lng': 102.9062154}","{'lat': 16.6874816, 'lng': 102.9060958}","{'lat': 16.6959559, 'lng': 102.911213}",None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,6,"{'lat': 16.0717586, 'lng': 102.7338184}","{'lat': 16.0521945, 'lng': 102.7206649}","{'lat': 16.067152, 'lng': 102.7323694}","{'lat': 16.0559465, 'lng': 102.7283399}","{'lat': 16.055911, 'lng': 102.7437213}","{'lat': 16.0706613, 'lng': 102.7329711}","{'lat': 16.0546789, 'lng': 102.7140396}","{'lat': 16.0731008, 'lng': 102.7368188}","{'lat': 16.0715606, 'lng': 102.7342584}",...,"{'lat': 16.0728641, 'lng': 102.733482}","{'lat': 16.0657114, 'lng': 102.7287431}","{'lat': 16.072814, 'lng': 102.7399438}","{'lat': 16.0560054, 'lng': 102.7258071}","{'lat': 16.0436701, 'lng': 102.7384617}","{'lat': 16.0637541, 'lng': 102.7209871}","{'lat': 16.0687492, 'lng': 102.7357992}","{'lat': 16.0484948, 'lng': 102.7470726}","{'lat': 16.0690776, 'lng': 102.7309927}","{'lat': 16.0697929, 'lng': 102.7340638}"
3,7,"{'lat': 16.532009, 'lng': 102.5913487}","{'lat': 16.5469506, 'lng': 102.5946326}","{'lat': 16.5304647, 'lng': 102.5888848}","{'lat': 16.5487389, 'lng': 102.5952108}",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,9,"{'lat': 15.8353887, 'lng': 102.6086849}","{'lat': 15.8540116, 'lng': 102.6297322}","{'lat': 15.8424063, 'lng': 102.6286231}","{'lat': 15.8353984, 'lng': 102.6087099}","{'lat': 15.835351, 'lng': 102.608366}","{'lat': 15.8353984, 'lng': 102.6087099}","{'lat': 15.8353984, 'lng': 102.6087099}","{'lat': 15.8352424, 'lng': 102.6083645}","{'lat': 15.8611798, 'lng': 102.6086101}",...,"{'lat': 15.8353984, 'lng': 102.6087099}",None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3538,30342P,"{'lat': 16.3623531, 'lng': 102.6050499}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3539,30358P,"{'lat': 16.048969, 'lng': 102.80816}","{'lat': 16.0542794, 'lng': 102.7920569}",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3540,30359P,"{'lat': 16.4241996, 'lng': 102.8015855}","{'lat': 16.4512645, 'lng': 102.8162773}","{'lat': 16.4496923, 'lng': 102.8157815}","{'lat': 16.4317487, 'lng': 102.8111629}","{'lat': 16.4215141, 'lng': 102.8077957}","{'lat': 16.4500026, 'lng': 102.8184128}","{'lat': 16.4519146, 'lng': 102.8186759}","{'lat': 16.4357091, 'lng': 102.8052259}","{'lat': 16.4481546, 'lng': 102.7874673}",...,"{'lat': 16.4450546, 'lng': 102.8199799}","{'lat': 16.4382674, 'lng': 102.8206531}","{'lat': 16.443305, 'lng': 102.792247}","{'lat': 16.4354382, 'lng': 102.8058774}","{'lat': 16.445288, 'lng': 102.7923799}","{'lat': 16.4437682, 'lng': 102.7964501}","{'lat': 16.4338027, 'lng': 102.8142827}","{'lat': 16.4214039, 'lng': 102.8074362}","{'lat': 16.442887, 'lng': 102.822015}",None
3541,30360P,"{'lat': 16.4810561, 'lng': 102.8317087}","{'lat': 16.4687957, 'lng': 102.8010787}","{'lat': 16.4969229, 'lng': 102.8329121}","{'lat': 16.4716564, 'lng': 102.8263428}","{'lat': 16.4903517, 'lng': 102.8135589}","{'lat': 16.4799106, 'lng': 102.8178279}","{'lat': 16.47801, 'lng': 102.8203845}","{'lat': 16.4647151, 'lng': 102.8247331}","{'lat': 16.4806252, 'lng': 102.8199806}",...,"{'lat': 16.4691493, 'lng': 102.8302428}","{'lat': 16.4697047, 'lng': 102.8302438}",None,None,None,None,None,None,None,None


In [ ]:
scid_changename['school_idno'].isna().sum()

0

In [ ]:
type(scid_changename)

pandas.core.frame.DataFrame

In [ ]:
scid_changename.astype(str)

,school_idno,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,1,"{'lat': 16.1311875, 'lng': 102.6504375}","{'lat': 16.1350641, 'lng': 102.6290474}","{'lat': 16.1482073, 'lng': 102.6207588}","{'lat': 16.1279195, 'lng': 102.6138295}",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,4,"{'lat': 16.6744524, 'lng': 102.9189581}","{'lat': 16.6934, 'lng': 102.926531}","{'lat': 16.682199, 'lng': 102.909279}","{'lat': 16.6876442, 'lng': 102.9062154}","{'lat': 16.6874816, 'lng': 102.9060958}","{'lat': 16.6959559, 'lng': 102.911213}",None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,6,"{'lat': 16.0717586, 'lng': 102.7338184}","{'lat': 16.0521945, 'lng': 102.7206649}","{'lat': 16.067152, 'lng': 102.7323694}","{'lat': 16.0559465, 'lng': 102.7283399}","{'lat': 16.055911, 'lng': 102.7437213}","{'lat': 16.0706613, 'lng': 102.7329711}","{'lat': 16.0546789, 'lng': 102.7140396}","{'lat': 16.0731008, 'lng': 102.7368188}","{'lat': 16.0715606, 'lng': 102.7342584}",...,"{'lat': 16.0728641, 'lng': 102.733482}","{'lat': 16.0657114, 'lng': 102.7287431}","{'lat': 16.072814, 'lng': 102.7399438}","{'lat': 16.0560054, 'lng': 102.7258071}","{'lat': 16.0436701, 'lng': 102.7384617}","{'lat': 16.0637541, 'lng': 102.7209871}","{'lat': 16.0687492, 'lng': 102.7357992}","{'lat': 16.0484948, 'lng': 102.7470726}","{'lat': 16.0690776, 'lng': 102.7309927}","{'lat': 16.0697929, 'lng': 102.7340638}"
3,7,"{'lat': 16.532009, 'lng': 102.5913487}","{'lat': 16.5469506, 'lng': 102.5946326}","{'lat': 16.5304647, 'lng': 102.5888848}","{'lat': 16.5487389, 'lng': 102.5952108}",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,9,"{'lat': 15.8353887, 'lng': 102.6086849}","{'lat': 15.8540116, 'lng': 102.6297322}","{'lat': 15.8424063, 'lng': 102.6286231}","{'lat': 15.8353984, 'lng': 102.6087099}","{'lat': 15.835351, 'lng': 102.608366}","{'lat': 15.8353984, 'lng': 102.6087099}","{'lat': 15.8353984, 'lng': 102.6087099}","{'lat': 15.8352424, 'lng': 102.6083645}","{'lat': 15.8611798, 'lng': 102.6086101}",...,"{'lat': 15.8353984, 'lng': 102.6087099}",None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3538,30342P,"{'lat': 16.3623531, 'lng': 102.6050499}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3539,30358P,"{'lat': 16.048969, 'lng': 102.80816}","{'lat': 16.0542794, 'lng': 102.7920569}",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3540,30359P,"{'lat': 16.4241996, 'lng': 102.8015855}","{'lat': 16.4512645, 'lng': 102.8162773}","{'lat': 16.4496923, 'lng': 102.8157815}","{'lat': 16.4317487, 'lng': 102.8111629}","{'lat': 16.4215141, 'lng': 102.8077957}","{'lat': 16.4500026, 'lng': 102.8184128}","{'lat': 16.4519146, 'lng': 102.8186759}","{'lat': 16.4357091, 'lng': 102.8052259}","{'lat': 16.4481546, 'lng': 102.7874673}",...,"{'lat': 16.4450546, 'lng': 102.8199799}","{'lat': 16.4382674, 'lng': 102.8206531}","{'lat': 16.443305, 'lng': 102.792247}","{'lat': 16.4354382, 'lng': 102.8058774}","{'lat': 16.445288, 'lng': 102.7923799}","{'lat': 16.4437682, 'lng': 102.7964501}","{'lat': 16.4338027, 'lng': 102.8142827}","{'lat': 16.4214039, 'lng': 102.8074362}","{'lat': 16.442887, 'lng': 102.822015}",None
3541,30360P,"{'lat': 16.4810561, 'lng': 102.8317087}","{'lat': 16.4687957, 'lng': 102.8010787}","{'lat': 16.4969229, 'lng': 102.8329121}","{'lat': 16.4716564, 'lng': 102.8263428}","{'lat': 16.4903517, 'lng': 102.8135589}","{'lat': 16.4799106, 'lng': 102.8178279}","{'lat': 16.47801, 'lng': 102.8203845}","{'lat': 16.4647151, 'lng': 102.8247331}","{'lat': 16.4806252, 'lng': 102.8199806}",...,"{'lat': 16.4691493, 'lng': 102.8302428}","{'lat': 16.4697047, 'lng': 102.8302438}",None,None,None,None,None,None,None,None


In [ ]:
scid_changetype = scid_changename['school_idno'].str.replace("P","")  

In [ ]:
scid_changetype

0           1
1           4
2           6
3           7
4           9
        ...  
3538    30342
3539    30358
3540    30359
3541    30360
3542    30362
Name: school_idno, Length: 3543, dtype: object

In [ ]:
school = pd.concat([scid_changetype,scid_changename], axis=1)

In [ ]:
school.drop(school.columns[2])
sc1=school.iloc[:,0:1]
sc2=school.iloc[:,2:]

In [ ]:
school = pd.concat([sc1,sc2], axis=1)
schoolfinal=school.rename(columns={'school_idno':'school_id'})
schoolfinal

,school_id,0,1,2,3,4,5,6,7,8,...,10,11,12,13,14,15,16,17,18,19
0,1,"{'lat': 16.1311875, 'lng': 102.6504375}","{'lat': 16.1350641, 'lng': 102.6290474}","{'lat': 16.1482073, 'lng': 102.6207588}","{'lat': 16.1279195, 'lng': 102.6138295}",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,4,"{'lat': 16.6744524, 'lng': 102.9189581}","{'lat': 16.6934, 'lng': 102.926531}","{'lat': 16.682199, 'lng': 102.909279}","{'lat': 16.6876442, 'lng': 102.9062154}","{'lat': 16.6874816, 'lng': 102.9060958}","{'lat': 16.6959559, 'lng': 102.911213}",None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,6,"{'lat': 16.0717586, 'lng': 102.7338184}","{'lat': 16.0521945, 'lng': 102.7206649}","{'lat': 16.067152, 'lng': 102.7323694}","{'lat': 16.0559465, 'lng': 102.7283399}","{'lat': 16.055911, 'lng': 102.7437213}","{'lat': 16.0706613, 'lng': 102.7329711}","{'lat': 16.0546789, 'lng': 102.7140396}","{'lat': 16.0731008, 'lng': 102.7368188}","{'lat': 16.0715606, 'lng': 102.7342584}",...,"{'lat': 16.0728641, 'lng': 102.733482}","{'lat': 16.0657114, 'lng': 102.7287431}","{'lat': 16.072814, 'lng': 102.7399438}","{'lat': 16.0560054, 'lng': 102.7258071}","{'lat': 16.0436701, 'lng': 102.7384617}","{'lat': 16.0637541, 'lng': 102.7209871}","{'lat': 16.0687492, 'lng': 102.7357992}","{'lat': 16.0484948, 'lng': 102.7470726}","{'lat': 16.0690776, 'lng': 102.7309927}","{'lat': 16.0697929, 'lng': 102.7340638}"
3,7,"{'lat': 16.532009, 'lng': 102.5913487}","{'lat': 16.5469506, 'lng': 102.5946326}","{'lat': 16.5304647, 'lng': 102.5888848}","{'lat': 16.5487389, 'lng': 102.5952108}",None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,9,"{'lat': 15.8353887, 'lng': 102.6086849}","{'lat': 15.8540116, 'lng': 102.6297322}","{'lat': 15.8424063, 'lng': 102.6286231}","{'lat': 15.8353984, 'lng': 102.6087099}","{'lat': 15.835351, 'lng': 102.608366}","{'lat': 15.8353984, 'lng': 102.6087099}","{'lat': 15.8353984, 'lng': 102.6087099}","{'lat': 15.8352424, 'lng': 102.6083645}","{'lat': 15.8611798, 'lng': 102.6086101}",...,"{'lat': 15.8353984, 'lng': 102.6087099}",None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3538,30342,"{'lat': 16.3623531, 'lng': 102.6050499}",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3539,30358,"{'lat': 16.048969, 'lng': 102.80816}","{'lat': 16.0542794, 'lng': 102.7920569}",None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3540,30359,"{'lat': 16.4241996, 'lng': 102.8015855}","{'lat': 16.4512645, 'lng': 102.8162773}","{'lat': 16.4496923, 'lng': 102.8157815}","{'lat': 16.4317487, 'lng': 102.8111629}","{'lat': 16.4215141, 'lng': 102.8077957}","{'lat': 16.4500026, 'lng': 102.8184128}","{'lat': 16.4519146, 'lng': 102.8186759}","{'lat': 16.4357091, 'lng': 102.8052259}","{'lat': 16.4481546, 'lng': 102.7874673}",...,"{'lat': 16.4450546, 'lng': 102.8199799}","{'lat': 16.4382674, 'lng': 102.8206531}","{'lat': 16.443305, 'lng': 102.792247}","{'lat': 16.4354382, 'lng': 102.8058774}","{'lat': 16.445288, 'lng': 102.7923799}","{'lat': 16.4437682, 'lng': 102.7964501}","{'lat': 16.4338027, 'lng': 102.8142827}","{'lat': 16.4214039, 'lng': 102.8074362}","{'lat': 16.442887, 'lng': 102.822015}",None
3541,30360,"{'lat': 16.4810561, 'lng': 102.8317087}","{'lat': 16.4687957, 'lng': 102.8010787}","{'lat': 16.4969229, 'lng': 102.8329121}","{'lat': 16.4716564, 'lng': 102.8263428}","{'lat': 16.4903517, 'lng': 102.8135589}","{'lat': 16.4799106, 'lng': 102.8178279}","{'lat': 16.47801, 'lng': 102.8203845}","{'lat': 16.4647151, 'lng': 102.8247331}","{'lat': 16.4806252, 'lng': 102.8199806}",...,"{'lat': 16.4691493, 'lng': 102.8302428}","{'lat': 16.4697047, 'lng': 102.8302438}",None,None,None,None,None,None,None,None


In [ ]:
for id in schoolfinal['school_id']:
 print(id)

1
4
6
7
9
11
16
17
18
19
2
21
22
23
24
25
26
28
29
3
31
32
33
35
36
38
491
493
494
495
496
497
498
499
508
509
51
511
512
513
515
517
518
519
52
521
522
523
524
525
526
527
528
529
531
532
533
534
535
536
537
538
539
54
541
542
543
545
546
547
548
549
55
551
552
553
554
555
557
558
56
561
562
563
564
565
566
567
568
569
57
571
572
573
574
575
576
577
578
579
58
581
582
583
584
585
586
587
588
589
59
591
592
593
594
595
596
597
598
601
602
603
604
605
606
607
608
609
61
611
612
613
614
615
616
617
618
619
62
621
622
623
624
625
626
627
628
629
631
632
633
634
635
636
637
638
639
64
641
642
643
644
645
646
647
648
649
65
651
652
653
654
655
656
657
658
659
66
661
662
663
664
665
666
667
668
669
67
671
672
673
674
675
676
677
678
679
68
681
682
683
684
685
686
687
688
689
69
691
692
693
694
695
696
697
698
699
701
702
703
704
705
706
707
708
709
71
711
712
713
714
715
716
717
718
719
72
721
722
758
759
76
761
1024
1033
1056
1057
1058
1061
1062
1063
1064
1065
1066
1067
1068
107
1071
1073
1